In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
tfds.disable_progress_bar()

from keras.preprocessing import image

In [ ]:
image_size = 224
batch_size = 128

In [ ]:
model = keras.Sequential([
#     keras.layers.AveragePooling2D(6, 3, input_shape=(200, 200, 3)),
#     keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=(image_size,image_size,3), activation='relu', padding='same'),
    keras.layers.MaxPool2D(2, 2),
#     keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(120, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

# model.fit(X_train, y_train, epochs=2, batch_size=32)

In [ ]:
import numpy as np
# from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
#     output = preprocess_input(img_array)
    output = img_array
    return(output)

def imageLoader(files, batch_size, labels, image_size):
    L = len(files) 
    while True:
        batch_start = 0
        batch_end = batch_size
        while batch_start < L:
            limit = min(batch_end, L)
            X = read_and_prep_images(files[batch_start:limit], img_height=image_size, img_width=image_size)
            Y = labels[batch_start:limit]
            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     
            batch_start += batch_size   
            batch_end += batch_size
            
def get_img_path(img_id):
    image_path = os.path.join("../input/dog-breed-identification/train",img_id+'.jpg')
    return image_path

from sklearn.model_selection import train_test_split

train_labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
Y = pd.get_dummies(train_labels['breed'])
labels = Y.values
img_paths = train_labels['id'].apply(get_img_path).values

train_images,test_images,train_labels,test_labels = train_test_split(img_paths,labels,test_size=0.1)            

x_test = read_and_prep_images(test_images, img_height=image_size, img_width=image_size)

model.fit_generator(imageLoader(train_images,batch_size, train_labels, image_size),epochs=5, validation_data = zip(x_test, test_labels)) #steps_per_epoch=..., 